In [2]:
import tensorflow as tf

In [ ]:
tf.initializers.GlorotUniform()

In [4]:
def conv2d(inputs, num_outputs,
          kernel_shape, # [kernel_height, kernel_stride]
          mask_type, # A = position of current pixel, B = from next of curr pixel to end of image
          strides=[1, 1], # [column_wise, row_wise]
          padding = 'SAME',
          activation_fn = None,
          weights_initializaer = tf.initializers.GlorotUniform(), # tf.contrib.layers.xavier_initializer(),
          weights_regularizer=None,
          biases_initializer=tf.zeros_initializer,
          biases_regularizer=None,
          scope='conv2d'):
    
    batch_size, height, width, channel = inputs.get_shape().as_list()
    kernel_h, kernel_w = kernel_shape
    stride_h, stride_w = strides
    
    center_h = kernel_h // 2
    center_w = kernel_w // 2
    
    weights_shape = [kernel_h, kernel_w, channel, num_outputs]
    weights = tf.get_variable("weights", weights_shape, 
                              tf.float32, weights_initializer, weights_regularizer)
    
    if mask_type is not None:
        mask = np.ones((kernel_h, kernel_w, channel, num_outputs), dtype=np.float32)

        mask[center_h, center_w+1: ,: ,:] = 0.
        mask[center_h+1:, :, :, :] = 0.

    if mask_type == 'a':
        mask[center_h,center_w,:,:] = 0.

    weights *= tf.constant(mask, dtype=tf.float32)
    tf.add_to_collection('conv2d_weights_%s' % mask_type, weights)
    
    outputs = tf.nn.conv2d(inputs, 
                           weights, [1, stride_h, stride_w, 1], padding=padding, name='outputs')
    tf.add_to_collection('conv2d_outputs', outputs)

    if biases_initializer != None:
        biases = tf.get_variable("biases", [num_outputs,], 
                                 tf.float32, biases_initializer, biases_regularizer)
        outputs = tf.nn.bias_add(outputs, biases, name='outputs_plus_b')

    if activation_fn:
        outputs = activation_fn(outputs, name='outputs_with_fn')
    return outputs

In [7]:
def pixelRNN(height, width, channel, params):
    """
    Args
    height, width, channel - the dimensions of the input
    params the hyperparameters of the network
    """
    input_shape = [None, height, width, channel]
    inputs = tf.placeholder(tf.float32, input_shape)
    
# ============================논문에서 제시된 모든 모델의 도입부====================

    # input of main recurrent layers
    scope = "conv_inputs"
    conv_inputs = conv2d(inputs, params.hidden_dims, [7, 7], "A", scope=scope)
                                # params.hidden_dims -> 128
# ===========================================================================
    
    # main recurrent layers
    last_hid = conv_inputs
    for idx in xrange(params.recurrent_length):
        scope = 'CONV%d' % idx
        last_hid = conv2d(last_hid, 3, [1, 1], "B", scope=scope)
        print("Building %s" % scope)
        
        
    # output recurrent layers
    for idx in xrange(params.out_recurrent_length):
        scope = 'CONV_OUT%d' % idx
        last_hid = tf.nn.relu(conv2d(last_hid, params.out_hidden_dims, [1, 1], "B", scope=scope))
                                            # params.out_hidden_dims -> 64
        print("Building %s" % scope)
        
        
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(conv2d_out_logits, inputs, name='loss'))

    optimizer = tf.train.RMSPropOptimizer(p.learning_rate)
    grads_and_vars = optimizer.compute_gradients(loss)

    new_grads_and_vars = \
        [(tf.clip_by_value(gv[0], -p.grad_clip, p.grad_clip), gv[1]) for gv in grads_and_vars]
    optim = optimizer.apply_gradients(new_grads_and_vars)

In [6]:
loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(conv2d_out_logits, inputs, name='loss'))

optimizer = tf.train.RMSPropOptimizer(p.learning_rate)
grads_and_vars = optimizer.compute_gradients(loss)

new_grads_and_vars = \
    [(tf.clip_by_value(gv[0], -p.grad_clip, p.grad_clip), gv[1]) for gv in grads_and_vars]
optim = optimizer.apply_gradients(new_grads_and_vars)

NameError: name 'conv2d_out_logits' is not defined

In [8]:
def predict(sess, images, inputs, output):
    return sess.run(output, {inputs: images})

def generate(sess, height, width, inputs, output):
    samples = np.zeros((100, height, width, 1), dtype='float32')

    for i in range(height):
        for j in range(width):
            next_sample = binarize(predict(sess, samples, inputs, output))
            samples[:, i, j] = next_sample[:, i, j]

    return samples

def generate_occlusions(sess, height, width, inputs, output):
    samples = occlude(images, height, width)
    starting_position = [0,height//2]
    for i in range(starting_position[1], height):
        for j in range(starting_position[0], width):
            next_sample = binarize(predict(sess, samples, inputs, output))
            samples[:, i, j] = next_sample[:, i, j]
    return samples